In [4]:
from ladybug.epw import EPW
from kvrrj.wind import Wind
from kvrrj.solar import Solar
import numpy as np
import pandas as pd

# riyadh
epw_file = r"C:\Users\tgerrish\OneDrive - Buro Happold\Weather files\epws\ARE_SH_Sharjah.Intl.AP.411960_TMYx.2007-2021.epw"
epw = EPW(epw_file)

wind = Wind.from_epw(epw_file)
solar = Solar.from_epw(epw_file)


In [12]:
from kvrrj.ladybug.util import _timezone_valid_for_lat_long, EPW

epw_file = r"C:\Users\tgerrish\OneDrive - Buro Happold\Weather files\epws\ARE_SH_Sharjah.Intl.AP.411960_TMYx.2007-2021.epw"
epw = EPW(epw_file)

_timezone_valid_for_lat_long(epw.location.latitude, epw.location.longitude, 28)

ValueError: ('absolute offset is too large', 1680)

In [15]:
epw.monthly_ground_temperature[0.5].filter_by_months([1, 2, 3]).values

(20.56, 21.8, 23.75)

In [ ]:
geo = ROOM.faces[1]
kwargs = {}

traces = []
kwargs["facecolor"] = to_plotly_color(to_ladybug_color(geo))
# check for sub-faces, and split geometry if true
if geo.has_sub_faces:
    traces.append(to_plotly(geo.punched_geometry, **kwargs))
else:
    traces.append(to_plotly(geo, **kwargs))
# print(ROOM.faces[0])
# fig = go.Figure()
# fig.add_trace(to_plotly(ROOM.faces[0]))
# # fig.add_trace(to_plotly(ROOM.faces))
# fig.show()
traces


In [ ]:
from honeybee.model import Wall, Face, Aperture, Room

from kvrrj.geometry.to_plotly import to_plotly
import plotly.graph_objects as go

# create a room
ROOM = Room.from_box(identifier="room")
# add apertures to 3 walls

shades = []
for n, face in enumerate(ROOM.faces):
    face: Face
    if isinstance(face.type, Wall):
        face.apertures_by_ratio(0.4)
        for aperture in face.apertures:
            aperture: Aperture
            shades.append(aperture.louvers_by_count(louver_count=3, depth=0.1))


to_plotly(ROOM.faces[1])
# # fig = go.Figure()
# # fig.add_traces()
# # fig.show()



ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter3d: 'color'

Did you mean "mode"?

    Valid properties:
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            `customdata`.
        error_x
            :class:`plotly.graph_objects.scatter3d.ErrorX` instance
            or dict with compatible properties
        error_y
            :class:`plotly.graph_objects.scatter3d.ErrorY` instance
            or dict with compatible properties
        error_z
            :class:`plotly.graph_objects.scatter3d.ErrorZ` instan

In [1]:
from ladybug_geometry.geometry3d import Face3D, Point3D, Mesh3D, Vector3D, Polyface3D, Plane
from kvrrj.geometry import to_shapely, to_plotly
import plotly.graph_objects as go

from honeybee.model import Door
geo = Door.from_vertices(identifier="asdfoijn", vertices=Face3D.from_regular_polygon(side_count=3).vertices)

fig = go.Figure()
fig.add_trace(to_plotly(geo))
fig.show()


In [ ]:
# msh = geo.triangulated_mesh3d
# xyz = [pt.to_array() for pt in msh.vertices]
# x, y, z = [list(i) for i in zip(*xyz)]
# i, j, k = [list(i) for i in zip(*msh.faces)]

# fig = go.Figure(
#     data=[
#         go.Mesh3d(
#     x=x,
#     y=y,
#     z=z,
#     i=i,
#     j=j,
#     k=k,
# )
#     ]
# )

# fig.show()


In [93]:
fig = go.Figure(
    data=[
        go.Mesh3d(
            x=[0, 1, 2, 0],
            y=[0, 0, 1, 2],
            z=[0, 2, 0, 1],
            colorbar=dict(title=dict(text="z")),
            colorscale=[[0, "gold"], [0.5, "mediumturquoise"], [1, "magenta"]],
            # Intensity of each vertex, which will be interpolated and color-coded
            intensity=[0, 0.33, 0.66, 1],
            # i, j and k give the vertices of triangles
            # here we represent the 4 triangles of the tetrahedron surface
            i=[0, 0, 0, 1],
            j=[1, 2, 3, 2],
            k=[2, 3, 1, 3],
            name="y",
            showscale=True,
        )
    ]
)

fig.show()


In [92]:
# base_url = "https://raw.githubusercontent.com/plotly/datasets/master/ply/"
# mesh_names = ["sandal", "scissors", "shark", "walkman"]
# dataframes = {name: pd.read_csv(base_url + name + "-ply.csv") for name in mesh_names}

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Mesh3d(x=dataframes["walkman"].x, 
          y=dataframes["walkman"].y,
          z=dataframes["walkman"].z,
          i=dataframes["walkman"].i,
          j=dataframes["walkman"].j,
          k=dataframes["walkman"].k
))
fig.show()

In [46]:
from kvrrj.geometry.to_plotly import (
    to_plotly,
    Point2D,
    Vector2D,
    Vector3D,
    Point3D,
    Arc2D,
    Arc3D,
    Plane,
    Ray2D,
    Ray3D,
    LineSegment2D,
    Polyline2D,
    Polyline3D,
    LineSegment3D,
    Face3D,
    Polygon2D
)
from kvrrj.geometry.to_shapely import to_shapely


import plotly.graph_objects as go
# import plotly.io as pio
# pio.renderers.default = "vscode"
from plotly.subplots import make_subplots

# creta efig with two subplots, one for 2D and one for 3D
# fig = go.Figure()
# fig.add_trace(to_plotly(Point2D(), marker={"color": "red"}))
# fig.add_trace(to_plotly(Arc2D(c=Point2D(), r=1), marker={"color": "blue"}))
# fig.add_trace(to_plotly(Ray2D(p=Point2D(), v=Vector2D(0, 1)), marker={"color": "green"}))
# fig.add_trace(to_plotly(LineSegment2D(p=Point2D(), v=Vector2D(1, 1)), marker={"color": "purple"}))
# fig.add_trace(to_plotly(Polyline2D.from_polygon(Polygon2D.from_regular_polygon(number_of_sides=5)), marker={"color": "orange"}))
# fig.show()

fig = go.Figure()
fig.add_trace(to_plotly(Point3D(), marker={"color": "red"}))
fig.add_trace(to_plotly(Arc3D(plane=Plane(), radius=1), marker={"color": "blue"}))
fig.add_trace(to_plotly(Ray3D(p=Point3D(), v=Vector3D(0, 0, 1)), marker={"color": "green"}))
fig.add_trace(to_plotly(LineSegment3D(p=Point3D(), v=Vector3D(1, 1, 0.5)), marker={"color": "purple"}))
fig.add_trace(
    to_plotly(
        Polyline3D.from_polyline2d(
            Polyline2D.from_polygon(Polygon2D.from_regular_polygon(number_of_sides=5))
        ),
        marker={"color": "orange"},
    )
)
fig.add_traces(to_plotly(
    Face3D(boundary=[Point3D(), Point3D(1, 0, 0), Point3D(1, 1, 0), Point3D(0, 1, 0)], holes=[[Point3D(0.25, 0.25, 0), Point3D(0.25, 0.5, 0), Point3D(0.5, 0.25)]]),
))
fig.show()


